# 5. LangChain解説

In [ ]:
import os

from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
!pip install langchain-core==0.1.18 langchain==0.1.5 langchain-openai==0.0.5 langchain_community==0.0.17

## 5-1 Data connection

### Document loaders

In [ ]:
!pip install GitPython==3.1.41

In [ ]:
from langchain_community.document_loaders import GitLoader

def file_filter(file_path):
    return file_path.endswith(".mdx")

loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

raw_docs = loader.load()
print(len(raw_docs))

### Document transformers

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

docs = text_splitter.split_documents(raw_docs)
print(len(docs))

## Text embedding models

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [ ]:
!pip install tiktoken==0.5.2

In [ ]:
query = "AWSのS3からデータを読み込むためのDocumentLoaderはありますか？"

vector = embeddings.embed_query(query)
print(len(vector))
print(vector)

### Vector stores

In [ ]:
!pip install chromadb==0.4.22

In [ ]:
from langchain_community.vectorstores import Chroma

db = Chroma.from_documents(docs, embeddings)

### Retriever

In [ ]:
retriever = db.as_retriever()

In [ ]:
query = "AWSのS3からデータを読み込むためのDocumentLoaderはありますか？"

context_docs = retriever.get_relevant_documents(query)
print(f"len = {len(context_docs)}")

first_doc = context_docs[0]
print(f"metadata = {first_doc.metadata}")
print(first_doc.page_content)

### RetrievalQA（Chain）

In [ ]:
# LCELでのRAGの実装例は以下のようになります。
#
# 参考: https://python.langchain.com/docs/expression_language/cookbook/retrieval

from langchain.chains import RetrievalQA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = PromptTemplate.from_template(
    """以下の文脈だけを踏まえて質問に回答してください。

{context}

Question: {question}
"""
)

model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

result = chain.invoke(query)
print(result)

## 5-2 Agents

### Agentsの使用例

In [ ]:
from langchain_core.globals import set_verbose

set_verbose(True)

In [ ]:
# terminalというツールを使うため、langchain-experimentalをインストール
# ReActによるAgentのプロンプトをダウンロードするため、langchainhubをインストール
!pip install langchain-experimental==0.0.50 langchainhub==0.1.14

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
tools = load_tools(["terminal"])
prompt = hub.pull("hwchase17/react")
agent = create_react_agent(chat, tools, prompt)

agent_chain = AgentExecutor(agent=agent, tools=tools)

result = agent_chain.invoke({"input": "sample_dataディレクトリにあるファイルの一覧を教えて"})
print(result["output"])

### Tools

In [ ]:
from langchain_core.tools import Tool

def my_super_func(param):
    return "42"

tools = [
    Tool.from_function(
        func=my_super_func,
        name="The_Answer",
        description="生命、宇宙、そして万物についての究極の疑問の答え"
    ),
]

In [ ]:
agent = create_react_agent(chat, tools, prompt)
agent_chain = AgentExecutor(agent=agent, tools=tools)

result = agent_chain.invoke({"input": "この世界の真理を教えてください"})
print(result["output"])

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

summarize_prompt = PromptTemplate.from_template(
    """以下の文章を結論だけ一言に要約してください。

{input}
"""
)

model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

summarize_chain = (
    {"input": RunnablePassthrough()}
    | summarize_prompt
    | model
    | StrOutputParser()
)

tools = [
    Tool.from_function(
        func=summarize_chain.invoke,
        name="Summarizer",
        description="Text summarizer"
    ),
]

In [ ]:
agent = create_react_agent(chat, tools, prompt)
agent_chain = AgentExecutor(agent=agent, tools=tools)

text = """以下を要約してください。

こんにちは！私はChatGPTと呼ばれるAI言語モデルです。OpenAIが開発したGPT-3.5アーキテクチャに基づいています。私は自然言語理解と生成に特化しており、さまざまなトピックに関する質問に答えたり、おしゃべりしたりすることが得意です。
私のトレーニングデータは2021年9月までの情報に基づいているため、それ以降の出来事については知識がありません。ですが、できる限りお手伝いすることに努めます。
質問や会話、情報の共有など、どんなお手伝いでもお気軽にお申し付けください！よろしくお願いします。"""

result = agent_chain.invoke({"input": text})
print(result["output"])

### Function callingを使うOpenAI Functions Agent

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
tools = load_tools(["terminal"])
prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_openai_tools_agent(chat, tools, prompt)
agent_chain = AgentExecutor(agent=agent, tools=tools)

result = agent_chain.invoke({"input": "sample_dataディレクトリにあるファイルの一覧を教えて"})
print(result["output"])

### 複数のツールを一度に使うOpenAI Multi Functions Agent

In [ ]:
!pip install duckduckgo-search==5.3.0

In [ ]:
from langchain_core.globals import set_debug, set_verbose

set_debug(True)
set_verbose(False)

In [ ]:
# Chat Completions APIのアップデートにより、「functions」ではなく「tools」を使用するのが推奨になりました。
# 「tools」では一度に複数のツールの呼び出しが可能なため、OpenAI Multi Functions Agentを使う必要はなくなりました。
# ただし、「tools」で一度に複数のツールが呼び出すようにするには、gpt-3.5-turbo-1106やgpt-3.5-turbo-0125といった新しいモデルを使う必要があります。

from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent, load_tools
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
tools = load_tools(["ddg-search"])
prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_openai_tools_agent(chat, tools, prompt)
agent_chain = AgentExecutor(agent=agent, tools=tools)

result = agent_chain.invoke({"input": "東京と大阪の天気を教えて"})
print(result["output"])

### （コラム）Function callingを応用したOurputParser・Extraction・Tagging

In [ ]:
from langchain_core.globals import set_debug, set_verbose

set_debug(False)
set_verbose(False)

In [ ]:
import json
from typing import Optional

from langchain.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field

class Person(BaseModel):
    person_name: str
    person_height: str
    person_hair_color: Optional[str]
    dog_name: Optional[str]
    dog_breed: Optional[str]

class People(BaseModel):
    people: list[Person]

model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0).bind_tools([People])

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful assistant"),
    ("user", "{input}")
])

chain = prompt | model | JsonOutputToolsParser()

text = """
Alex is 5 feet tall. Claudia is 1 feet taller Alex and jumps higher than him. Claudia is a brunette and Alex is blonde.
Alex's dog Frosty is a labrador and likes to play hide and seek.
"""

people = chain.invoke({"input": text})
print(json.dumps(people, indent=2))

## まとめ

### （コラム）Evaluation

In [ ]:
from langchain.evaluation import load_evaluator
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-4", temperature=0)

evaluator = load_evaluator("qa", eval_llm=chat)

result = evaluator.evaluate_strings(
    input="私は市場に行って10個のリンゴを買いました。隣人に2つ、修理工に2つ渡しました。それから5つのリンゴを買って1つ食べました。残りは何個ですか？",
    prediction="""1最初に10個のリンゴを買い、その中から隣人と修理工にそれぞれ2個ずつ渡しました。そのため、まず手元に残ったリンゴは10 - 2 - 2 = 6個となります。

その後、さらに5個のリンゴを買い、1つ食べました。これにより手元のリンゴは6 + 5 - 1 = 10個となります。""",
    reference="10個",
)

print(result)

上記の入力は [Chain-of-Thoughtプロンプティング | Prompt Engineering Guide](https://www.promptingguide.ai/jp/techniques/cot) から引用しました。